# Prueba 1: Análisis de Sentimientos de Twitter (Sebastián Rebolledo)

## Hito 1: Aspectos Computacionales

### Preliminares

#### En el siguiente ejercicio se realizará la clasificación de Tweet en positivos o negativos en funcion de los sentimientos que expresa. Para ello se implementarán distintos modelos de clasificación y se evaluara su desempeño.

- Las metricas que se utilizarán para medir el desempeño de los modelos serán Accuracy, precision y Recall. 
- Se realizará primeramente el preprocesamiento de las variables del vector sentiment, quedando esta variable con 0 y 1, donde 0 serán tweets con sentimientos negativos y 1 tweets con sentimientos positivos.
- Luego se realizará un preprocesamiento de palabras en el atributo 'content' utilizando CountVectorizarer.

### Modelos a implementar

- Regresion logistica.

- LinearDiscriminantAnalysis.

- Árbol de clasificación, en donde se implementara una grilla para buscar los mejores hiperparametros. A continuacion grilla tentativa.
    - params = {'max_depth':[10,30,50,70], }
    
- GradientBoostingClasifiers : Se implementara una grilla para buscar los mejores hiperparametros. A continuacion grilla tentativa.
    - params = {'learning_rate': [0.01, 0.1, 0.5],'n_estimators': [50, 100, 500, 1000, 2000], 'subsample': [0.1,0.5,0.9]}.
                   
- AdaBoostClassifier: Se implementara una grilla para buscar los mejores hiperparametros. A continuacion grilla tentativa.
     - params = {'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [50, 100, 500, 1000, 2000]}

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mi_modulo as fn
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


NameError: name 'nltk' is not defined

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('wordnet')

In [ ]:
df = pd.read_csv('training_tweets.csv').drop('Unnamed: 0', axis=1)

In [ ]:
df.info()

No se encuentran datos perdidos.

In [ ]:
df.sentiment.value_counts()

A continuacion se recodificara el vector sentiment con un 0 para las emociones negativas y con un 1 para las emociones positivas.

In [ ]:
fn.recod_sentiment(df,['worry', 'sadness', 'hate', 'empty', 'boredom', 'anger'], 0)

In [ ]:
fn.recod_sentiment(df,['happiness', 'love', 'surprise', 'fun', 'relief', 'enthusiasm'], 1)

Dentro del atributo sentiment, se encuentran tweets con la denominacion de 'neutral', lo que significa que no esta claro si son atributos positivos o negativos, por cual serán repartidos al azar entre positivos y negativos.

In [ ]:
for index, valor in df[df['sentiment']== 'neutral']['content'].iteritems():
    #print(f'{index}-{valor}')
    df['sentiment'][index]=len(valor)%2

In [ ]:
df.sentiment.value_counts()

In [ ]:
df['sentiment'] = df['sentiment'].astype('int')

#### Distribucion del vector objetivo

In [ ]:
sns.countplot(x='sentiment', data=df)
plt.title('Comportamiento del atributo sentiment')
plt.show()

Se observa que hay una distribución de valores similar entre ambas clases.

In [ ]:
df

### Lowercase

In [ ]:
df.content = df.content.str.lower()

In [ ]:
df.content 

### Removiendo caracteres

In [ ]:
for index, row in df.iterrows():
    df['content'][index]= re.sub(r"(@[A-Za-z0-9]+)","", row['content'])

In [ ]:
for index, row in df.iterrows():
    df['content'][index]= re.sub("[^a-zA-Z]"," ", row['content'])

In [ ]:
df.content

In [ ]:
df.content[1]

### Tokenización

In [ ]:
for index, row in df.iterrows():
    df['content'][index]= nltk.tokenize.word_tokenize(df['content'][index])

In [ ]:
df.content

### Stopwords

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))
def clean_stopwords(token):
    return [item for item in token if item not in stop_words]

In [ ]:
df['content'] = df['content'].apply(clean_stopwords)

In [ ]:
df['content']

### Lemmatization

In [ ]:
lemma = nltk.stem.WordNetLemmatizer()
def clean_lemmatization(token):
    return [lemma.lemmatize(word=w, pos='v') for w in token]

In [ ]:
df['content'] = df['content'].apply(clean_lemmatization)

In [ ]:
df['content']

In [ ]:
def convertir_a_str(lista_tokens):
    return ' '.join(lista_tokens)

In [ ]:
df['content'] = df['content'].apply(convertir_a_str)

In [ ]:
df

In [ ]:
sentiment_1 = df[df['sentiment']==1]['content']

In [ ]:
sentiment_2 = df[df['sentiment']==2]['content']

### Conjunto de prueba y validación

In [ ]:
X=df['content']
y=df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.30, random_state=13213)

### Vectorizar

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit(X_train)

In [ ]:
tfi_train = vectorizer.transform(X_train)

In [ ]:
tfi_test = vectorizer.transform(X_test)

#### Regresion Logistica

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(tfi_train, y_train)

In [ ]:
y_hat_lr = lr.predict(tfi_test)

In [ ]:
print(classification_report(y_test, y_hat_lr))

Se observa que las clases estan bien distribuidas, que el valor de precision de ambas clases es el mismo, variando solamente en el recall, donde es mayor para la clase 0.

### Análisis Cuadrático Discriminante

In [ ]:
qda_model = QuadraticDiscriminantAnalysis().fit(tfi_train, y_train)

In [ ]:
y_hat_lda = qda_model.predict(tfi_test)

In [ ]:
print(classification_report(y_test, y_hat_qda))

#### Arbol de Clasificacion

In [ ]:
tree_para = {'max_depth':[50,100,200,250], 'max_features' : [None, 'log2', 'sqrt']}
tree_model_grid = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5, n_jobs=-1).fit(tfi_train, y_train)

In [ ]:
tree_model_grid.best_params_

In [ ]:
y_hat_tree_model = tree_model_grid.best_estimator_.predict(tfi_test)

In [ ]:
y_hat_tree_model

In [ ]:
print(classification_report(y_test, y_hat_tree_model))

***Se genera un error por que el modelo devuelve valores continuos.Aun no entiendo por que sucede. Se corregirá para la proxima semana.***

#### AdaBoostClassifier

In [ ]:
params_ada={'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [50,100, 500, 1000, 2000]}
ada_grid_cv = GridSearchCV(AdaBoostClassifier(), params_ada, n_jobs = -1).fit(tfi_train, y_train)

In [ ]:
ada_grid_cv.best_params_

In [ ]:
y_hat_ada = ada_grid_cv.best_estimator_.predict(tfi_test)

In [ ]:
print(classification_report(y_test, y_hat_ada))

Se observa que para la clase 1 la precision es del 70% y el recall de 57%, lo que indica que el modelo es capaz de identificar las caracteristicas más marcadas de la clase, pero no todas sus caracteristicas.

#### GradientBoostingClassifier

In [ ]:
param_gradient = {'learning_rate': [0.01, 0.1, 0.5],'n_estimators': [50, 100, 500, 1000, 2000], 
                   'subsample': [0.1,0.5,0.9]}
gradient_grid_cv = GridSearchCV(GradientBoostingClassifier(), param_gradient, n_jobs=-1, 
                                cv=5).fit(tfi_train, y_train)

In [ ]:
gradient_grid_cv.best_params_

In [ ]:
y_hat_gradient_boost=gradient_grid_cv.best_estimator_.predict(tfi_test)

In [ ]:
print(classification_report(y_test, y_hat_gradient_boost))

Se observa un modelo con resultados similares al modelo realizado con AdaBoost